In [1]:
import warnings 
warnings.filterwarnings('ignore')

In [2]:
#importing dependencies
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import shutil
import glob

In [3]:
# count the number of images in Diabetic and Non Diabetic sets
ROOT_DIR="D:\Internship Code Clause\Diabetic-Retinopathy-using-CNN-and-Transfer-Leaning\dataset"

number_of_images={}
for dir in os.listdir(ROOT_DIR):
    number_of_images[dir]=len(os.listdir(os.path.join(ROOT_DIR,dir)))

In [5]:
number_of_images.items()

dict_items([('DR', 1592), ('No_DR', 1548)])

In [6]:
# we will split the data such that 70% train data and 15% validataion and 15% test data

In [7]:
#Function to create the train,test,validation sets
def foldercreator(path,split):
    if not os.path.exists("./"+path):
        os.mkdir("./"+path)
        for dir in os.listdir(ROOT_DIR):
            os.makedirs("./"+path+"./"+dir)
            for img in np.random.choice(a=os.listdir(os.path.join(ROOT_DIR,dir)),size=(math.floor(split*number_of_images[dir])-5),replace=False):
                o=os.path.join(ROOT_DIR,dir,img)# path
                D=os.path.join("./"+path,dir)
                shutil.copy(o,D)
                os.remove(o)

    else:
        print("The folder exsist")

In [8]:
foldercreator("train",0.7)
foldercreator("test",0.15)
foldercreator("validation",0.15)

In [9]:
#Model Building 
from keras.layers import Conv2D,MaxPool2D,Dropout,Flatten,Dense,BatchNormalization,GlobalAvgPool2D
from keras.models import Sequential
from keras_preprocessing.image import load_img,img_to_array

In [10]:
#CNN MODEL
model=Sequential()
model.add(Conv2D(filters=16,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
model.add(Conv2D(filters=36, kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=64, kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(units=64,activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(units=1,activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 16)      448       
                                                                 
 conv2d_1 (Conv2D)           (None, 220, 220, 36)      5220      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 110, 110, 36)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 108, 108, 64)      20800     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 52, 52, 128)       7

In [11]:
import keras.losses

In [12]:
model.compile(optimizer='adam',loss=keras.losses.binary_crossentropy, metrics=['accuracy'])

In [13]:
#Preparing our data using data generator
from keras.preprocessing.image import  ImageDataGenerator

In [14]:
def preprocessingImages(path):
 """
 input : Path
 output :Preprocessed images
 
 """
 image_data=ImageDataGenerator(zoom_range=0.2,shear_range=0.2,rescale=1/255, horizontal_flip=True)
 image=image_data.flow_from_directory(directory=path,target_size=(224,224),batch_size=32,class_mode='binary')

 return image

In [15]:
path='train'
train_data=preprocessingImages(path)

Found 2187 images belonging to 2 classes.


In [16]:
def preprocessingImages_validate(path):
 """
 input : Path
 output :Preprocessed images
 
 """
 image_data=ImageDataGenerator(rescale=1/255)
 image=image_data.flow_from_directory(directory=path,target_size=(224,224),batch_size=32,class_mode='binary')

 return image

In [17]:
test_path='test'
test_data=preprocessingImages_validate(test_path)

Found 460 images belonging to 2 classes.


In [18]:
validate_path='validation'
validation_data=preprocessingImages_validate(validate_path)

Found 460 images belonging to 2 classes.


In [24]:
from keras.callbacks import ModelCheckpoint,EarlyStopping
#early stopping 
es=EarlyStopping(monitor="accuracy", min_delta=0.01, patience=20, verbose=1,mode='auto')
#model checkpoint 
mc=ModelCheckpoint(monitor="accuracy", filepath='./bestmodel.h5',verbose=1,save_best_only=True,mode='auto')
cd=[es,mc]

In [25]:
#Model Training
hs=model.fit_generator(generator=train_data,steps_per_epoch=8, epochs=50,verbose=1,validation_data=validation_data,validation_steps=16,
                       callbacks=cd)

Epoch 1/50
8/8 [==============================] - ETA: 0s - loss: 0.2167 - accuracy: 0.9180
Epoch 1: accuracy improved from -inf to 0.91797, saving model to .\bestmodel.h5
8/8 [==============================] - 14s 2s/step - loss: 0.2167 - accuracy: 0.9180
Epoch 2/50
8/8 [==============================] - ETA: 0s - loss: 0.2022 - accuracy: 0.9180
Epoch 2: accuracy did not improve from 0.91797
8/8 [==============================] - 14s 2s/step - loss: 0.2022 - accuracy: 0.9180
Epoch 3/50
8/8 [==============================] - ETA: 0s - loss: 0.1962 - accuracy: 0.9180
Epoch 3: accuracy did not improve from 0.91797
8/8 [==============================] - 14s 2s/step - loss: 0.1962 - accuracy: 0.9180
Epoch 4/50
8/8 [==============================] - ETA: 0s - loss: 0.1990 - accuracy: 0.9258
Epoch 4: accuracy improved from 0.91797 to 0.92578, saving model to .\bestmodel.h5
8/8 [==============================] - 14s 2s/step - loss: 0.1990 - accuracy: 0.9258
Epoch 5/50
8/8 [===================

In [26]:
# Model Accuracy
from keras.models import load_model
model=load_model("bestmodel.h5")

In [27]:
acc=model.evaluate_generator(test_data)[1]
print(acc*100)

93.9130425453186


In [32]:
#Transfer Learning
import  numpy as np
import matplotlib.pyplot as plt
from keras.layers import Flatten,Dense
from keras.models import Model,load_model
from keras.applications.mobilenet import MobileNet,preprocess_input

In [33]:
def preprocessingImages_transfer(path):
 """
 input : Path
 output :Preprocessed images
 
 """
 image_data=ImageDataGenerator(zoom_range=0.2,shear_range=0.2,preprocessing_function=preprocess_input, horizontal_flip=True)
 image=image_data.flow_from_directory(directory=path,target_size=(224,224),batch_size=32,class_mode='binary')

 return image

In [34]:
path='train'
train_data_transfer=preprocessingImages_transfer(path)

Found 2187 images belonging to 2 classes.


In [35]:
def preprocessingImages_validate_transfer(path):
 """
 input : Path
 output :Preprocessed images
 
 """
 image_data=ImageDataGenerator(preprocessing_function=preprocess_input)
 image=image_data.flow_from_directory(directory=path,target_size=(224,224),batch_size=32,class_mode='binary')

 return image

In [36]:
test_path='test'
test_data_transfer=preprocessingImages_validate_transfer(test_path)

Found 460 images belonging to 2 classes.


In [37]:
validate_path='validation'
validation_data_transfer=preprocessingImages_validate_transfer(validate_path)

Found 460 images belonging to 2 classes.


In [38]:
base_model=MobileNet(input_shape=(224,224,3),include_top=False)

In [39]:
for layers in base_model.layers:
    layers.trainable=False

In [40]:
base_model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                         

In [41]:
X=Flatten()(base_model.output)
X=Dense(units=1, activation='sigmoid')(X)
model_transfer=Model(base_model.input,X)

In [42]:
model_transfer.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                      

In [43]:
model_transfer.compile(optimizer='rmsprop',loss=keras.losses.binary_crossentropy,metrics=['accuracy'])

In [48]:
#Early Stopping and model checkpoint
from keras.callbacks import ModelCheckpoint,EarlyStopping
#early stopping
es=EarlyStopping(monitor='accuracy',min_delta=0.01, patience=10,verbose=1, mode='auto')
#Model Checkpoint
mc=ModelCheckpoint(monitor='accuracy',filepath="./bestmodel.h5", verbose=1,save_best_only=True,mode='auto')

cd=[es,mc]

In [49]:
his_transfer=model_transfer.fit_generator(generator=train_data_transfer,steps_per_epoch=8, epochs=30,verbose=1,validation_data=validation_data_transfer,validation_steps=16,
                       callbacks=cd)

Epoch 1/30
8/8 [==============================] - ETA: 0s - loss: 0.4590 - accuracy: 0.9609
Epoch 1: accuracy improved from -inf to 0.96094, saving model to .\bestmodel.h5
8/8 [==============================] - 5s 636ms/step - loss: 0.4590 - accuracy: 0.9609
Epoch 2/30
8/8 [==============================] - ETA: 0s - loss: 0.2424 - accuracy: 0.9648
Epoch 2: accuracy improved from 0.96094 to 0.96484, saving model to .\bestmodel.h5
8/8 [==============================] - 5s 649ms/step - loss: 0.2424 - accuracy: 0.9648
Epoch 3/30
8/8 [==============================] - ETA: 0s - loss: 0.0904 - accuracy: 0.9844
Epoch 3: accuracy improved from 0.96484 to 0.98438, saving model to .\bestmodel.h5
8/8 [==============================] - 5s 608ms/step - loss: 0.0904 - accuracy: 0.9844
Epoch 4/30
8/8 [==============================] - ETA: 0s - loss: 0.9358 - accuracy: 0.9297
Epoch 4: accuracy did not improve from 0.98438
8/8 [==============================] - 5s 646ms/step - loss: 0.9358 - accuracy

In [50]:
# Hence, The model trained with the efficiency of 98.438